In [ ]:
import json
import requests
import csv
import pandas as pd
import matplotlib.pyplot as plt
from ast import literal_eval
import numpy as np
import datetime

In [1]:
#Function to retrieve question from StackOverflow using the StackExchange API
def get_all_items(tag):
    base_url = 'https://api.stackexchange.com/2.3/questions'
    page = 1
    has_more = True
    all_items = []


    while has_more:
        params = {'page': page, 'pagesize': 100, 'order':'desc','sort':'activity','tagged':tag, 'site':'stackoverflow',
                  'filter':'!67FLdRRZSxs2Fsn5eSX(_u4Pa)7QBCJmP-nCWIM1*fW2NjxHGzaul9a7c3m','key':'1w)qZgiYmE2CpM2JH1dfqA((' }  # Adjust 'pagesize' as needed
        response = requests.get(base_url, params=params)

        if response.status_code == 200:
            data = response.json()
            all_items.extend(data['items'])
            has_more = data['has_more']
            page += 1
        else:
            error_response = response.json()
            print(f"Request failed with status code {response.status_code} page number: {page} " )
            print("Error:", error_response)
            break

    return all_items

In [ ]:
#Function to create a csv file from the returned JSON object
def createCsv(csv_file_name):
  # Get all unique keys from the JSON items
  field_names = set().union(*(d.keys() for d in all_items))


  # Writing to CSV
  with open(csv_file_name, 'w', newline='', encoding='utf-8') as csv_file:
      writer = csv.DictWriter(csv_file, fieldnames=field_names)

      # Write the headers
      writer.writeheader()
    
      # Write each JSON item as a row in the CSV file
      for item in all_items:
          writer.writerow(item)

  print(f"CSV file '{csv_file_name}' has been created successfully.")

In [ ]:
all_items = get_all_items('huggingface')
createCsv('huggingface.csv')
all_items = get_all_items('huggingface-evaluate')
createCsv('huggingface-evaluate.csv')
all_items = get_all_items('huggingface-hub')
createCsv('huggingface-hub.csv')
all_items = get_all_items('huggingface-trainer')
createCsv('huggingface-trainer.csv')
all_items = get_all_items('huggingface-transformers')
createCsv('huggingface-transformers.csv')
all_items = get_all_items('huggingface-tokenizers')
createCsv('huggingface-tokenizers.csv')
all_items = get_all_items('huggingface-datasets')
createCsv('huggingface-datasets.csv')

In [2]:
#Function to combine all the files, each file belongs to a HF-related tag
def combine_csv_files(file_list, key_column):
    # Read and concatenate all CSV files
    df_list = [pd.read_csv(file) for file in file_list]
    combined_df = pd.concat(df_list, ignore_index=True)

    # Drop duplicates based on the key column
    combined_df.drop_duplicates(subset=[key_column], inplace=True)

    return combined_df

In [ ]:
# Here we check for duplicates
has_duplicates = concatenated_df.duplicated(subset='question_id').any()

if has_duplicates:
    print("Duplicates exist in the concatenated DataFrame.")
else:
    print("No duplicates found in the concatenated DataFrame.")

In [ ]:
concatenated_df.to_csv('HF_SO_2019_2024.csv', index=False)

In [ ]:
#preprocessing (convert UnixTime to DateTime)

df = pd.read_csv('HF_SO_2019_2024.csv')
df.rename(columns={'creation_date': 'creation_date_unix'}, inplace=True)
df.to_csv('HF_SO_2019_2024.csv', index=False)

In [ ]:
def toDateFromUnix(column,dataset,fileName):
  dataset[column] = pd.to_datetime(dataset[column], unit='s').dt.strftime('%Y-%m-%d')
  dataset.to_csv(fileName, index=False)

In [ ]:
toDateFromUnix('creation_date',dataset,'combined_huggingfaceTags_no_duplicates.csv')
toDateFromUnix('last_activity_date',dataset,'combined_huggingfaceTags_no_duplicates.csv')